In [1]:
import pandas as pd
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob 

import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import glob
import json
from random import sample
import sklearn
import re
import string
import warnings
from bs4 import BeautifulSoup
import gensim
from gensim.models.phrases import Phrases
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.old_text_preproc import preprocess_e2e, preprocess_pre_tokenizing
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata
import datetime
import os
import time
import sys
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from analysis_func.text_preproc import preproc_text
from analysis_func.topic_preproc import prepare_corp
from analysis_func.topic_model import topicmodel
from analysis_func.saveload_topicmodels import save_modelanddata, load_modelanddata


from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 128 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
climate_data = pd.read_csv("/data/shruti/conspirecovery/reddit/initial/smallConsp/climate_data.csv", header=0)
climate_data.head()

,Unnamed: 0,author,subreddit,created_utc,score,id,body,author_flair_text,type,clean_text,cleaner_text
0,316,ThrombBone,climateskeptics,1545115904,1.0,ec14vna,Your models are not proof and your revisionist...,NaN,comment,your models are not proof and your revisionist...,models proof revisionist history ethical
1,333,thesnarkisaboojum,climateskeptics,1365866358,4.0,c9eewk3,Everything was pretty happy during the medieva...,NaN,comment,everything was pretty happy during the medieva...,pretty happy medieval warm period edit len...
2,482,pr-mth-s,climateskeptics,1474383908,10.0,53ns5l,Most of Ben &amp; Jerry's 300K twitter followe...,NaN,submission,most of ben jerry s 300k twitter followers w...,ben jerry 300k twitter followers flavors lo...
3,510,bugsbunny4pres,climateskeptics,1541689831,10.0,9vas5k,Climate Genocide Coming,NaN,submission,climate genocide coming,climate genocide coming
4,662,barttali,climateskeptics,1501193648,4.0,dktddwn,Did you notice that NASA said **AVERAGE SEPTEM...,NaN,comment,did you notice that nasa said average septembe...,notice nasa average september extent word s...


In [3]:
pd.set_option("max_colwidth", -1)
climate_data[['body','cleaner_text']].sample(10)

<ipython-input-3-e2741d60f5c6>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("max_colwidth", -1)


,body,cleaner_text
236032,Well I wouldn't mind Disneyland being slightly closer.,wouldn mind disneyland closer
38820,"He got what he wanted and it did not reflect the information that was available about that time period. Stacked data is not science, even a person as wholly devoted to this scam like yourself would admit that it was amazing that the temperature record utilised to reflect the UK was slightly lower than todays period temperatures and yet they were cultivating wine. This is clearly inconsistent with the temperatures required in that region and thus, he got what he wanted and stopped looking to actually validate his results.\n\n\nIt is sloppy and inconsistent with the scientific method and highlights the sham that is your chosen career. Yamal data was inconsistent except for the magic 63, MWP existed and to a greater extent claimed by Mann and he is wrong. You know he is wrong, the Pope knows he was wrong and Mann knows he was wrong. \n\n\nLove the lies claim. Always with misrepresentations and then accusations against the person you are arguing with, Classic Alinsky and also a clear indicator of a tactic employed when you are LOSING the argument. Willie Soon is right, regardless of his sources, Mann is wrong regardless of who agrees with him. \n\n\nNo go away and write up an article on how global warming is causing lag in computer games or some other impossible claim. By the way, King Arthur chopped off the Black Knights limbs. No shooting involved, like everything else you have posted, you have wrapped a big lie in a small truth and claimed victory. When you stop lying, you may actually be able to do some science rather than the poor regurgitation of failed hacks.",wanted reflect time period stacked data science person wholly devoted scam admit amazing temperature record utilised reflect uk lower todays period temperatures cultivating wine inconsistent temperatures required region wanted stopped validate sloppy inconsistent scientific method highlights sham chosen career yamal data inconsistent magic 63 mwp existed greater extent claimed mann wrong wrong pope wrong mann wrong love lies claim misrepresentations accusations person arguing classic alinsky clear indicator tactic employed losing argument willie sources mann wrong agrees write article global warming causing lag computer games impossible claim king arthur chopped black knights limbs shooting involved posted wrapped big lie small truth claimed victory lying science poor regurgitation failed hacks
173789,Maybe the lack of meat is changing the brains of climate alarmists.,lack meat changing brains climate alarmists
19768,"&gt; I’ve seen the effects..\n\nCan you tell us what effects you have seen? I don't mean what you've read about but what you have experienced personally. Tell us mouth breathers how global warming harmed you personally in a real and physical way. Here's some examples:\n\n* I have to use scuba gear to get to my house now because, sea level rise.\n* I went to the beach but acid oceans had dissolved the sand.\n* I have to get my morning paper before dawn because it will catch on fire after sunrise.\n* Extreme drought-floods ruin almost every weekend outdoor activity now.\n* Right now I'm driving my SUV in a climate refugee caravan to Antarctica.\n* The 6th extinction killed all my houseplants.",i’ve effects effects don ve read experienced personally mouth breathers global warming harmed personally real physical examples scuba gear house sea level rise beach acid oceans dissolved sand morning paper dawn catch fire sunrise extreme drought floods ruin weekend outdoor activity driving suv climate refugee caravan antarctica 6th extinction killed houseplants
53788,"&gt; [There is a great deal of misinformation floating around the ""skeptic"" blogosphere along the lines of ""Svensmark's claims are being validated"", and this simply isn't so. He has been shown to be wrong about basically every non-controversial aspect of his ideas. That GCRs can hav

In [4]:
tuples = pd.read_csv("../lite_data/climate_tuples.csv", header=0)
tuples.head()
vaxAuthors = list(set(tuples['author'].tolist()))
dlist = []
fnames = glob.glob("/data/shruti/conspirecovery/reddit/initial/content/authorContent*.gz")
for f in fnames:
    print(f)
    temp_F = pd.read_csv(f, header=0)
    temp_F = temp_F.loc[temp_F['author'].isin(vaxAuthors)]
    dlist.append(temp_F)

data = pd.concat(dlist)
data.head()

/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000000.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000001.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000002.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000003.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000004.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000005.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000006.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000007.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000008.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000009.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000010.gz
/data/shruti/conspirecovery/reddit/initial/content/authorContent000000000011.gz
/data/shruti/conspirecovery/reddit/initi

,author,subreddit,created_utc,score,id,body,author_flair_text,type
3,jeepdave,todayilearned,1565252595,1.0,ewav3lm,Doesn't make the courts right.,NaN,comment
4,kriegson,progun,1539875448,5.0,e803e5h,"They always try to deflect, dismiss and de-person veritas and you'll find people will *refuse* to watch a single minute of the video while regurgitating MSM talking points over and over. \n\nHence the NPC meme.",NaN,comment
6,flowirin,pics,1485329315,1.0,dcvnbrj,"yep, waaay less here",NaN,comment
7,nakedapedude,newzealand,1483421680,6.0,dbxn157,"Swap his vegemite for marmite, that's pretty much the worst prank you can pull on a kiwi.",NaN,comment
8,ranman1124,MapPorn,1468941216,2.0,d5i7vpc,No.,NaN,comment


In [5]:
antivax = ["vaccineskeptics","VaccinesCause","HealthConspiracy", "VaxTalk", "VaccineIngredients"]
climateskeptic = ["climateskeptics","greenskeptics","climatechangebullshit","climatehoax","Globalwarmingisfake","climateconspiratard"]
chemprints = ["CHEMPRINTS","Chemtrail","chemtrails","chemtrailpilots"]
flatearth = ["notaglobe","theworldisflat","flatearth", "Flat_Earth", "Flatearther"]
proclimate = ["ClimateCrisis", "CarbonCapture", "Climate_Discussion", "Climate", "ClimateActionPlan", "ClimateOffensive", "climatechange"]
conspiracy = ["conspriacy", "The_Donald"]
politics = ["Libertarian", "Conservative"]

In [6]:
def subredditgroup(subreddit):
    if subreddit in proclimate:
        return "proclimate"
    if subreddit in climateskeptic:
        return "climateskeptic"
    if subreddit in conspiracy:
        return "conspiracysub"
    if subreddit in politics:
        return "politicssub"
    
    return subreddit

In [7]:
data['g_sub'] = data["subreddit"].apply(lambda x: subredditgroup(x))

In [8]:
subgroups = ["proclimate", "climateskeptic", "conspiracysub", "politicssub"]

In [9]:
nonclimate_data = data.loc[data['g_sub'].isin(subgroups)]

In [10]:
swords = []
with open("../lite_data/stopwords.txt", "r") as rfile:
    for line in rfile:
        swords.append(line.strip("\n"))

In [11]:
len(nonclimate_data)

1475681

In [12]:
nonclimate_data = nonclimate_data.dropna(subset=['body'])
len(nonclimate_data)

1475674

In [13]:
terms = ["ozone layer scam", "climate doomsday", "cult of climatology", "alarmist", 
         "carbon tax scam", "greenies", "virtue-signal", "virtue signal", "harmless gas", "climate politics", 
         "global warming scam", "climate alarmist", "warmist", "climate geniuses", "climate change alarmism", 
         "global warming 'problem'", "climategate", "climate terrorist", "climate hoax", "climatehoax", 
         "cultishly", "gloom narrative", "warming hoax", "climate alarmism"]

def is_present(text):
    try:
        for t in terms:
            if t in text.lower():
                return 1
    except:
        fakevar=1
        
nonclimate_data['denials'] = nonclimate_data['body'].parallel_apply(lambda x: is_present(x))
nonclimate_data.loc[nonclimate_data['denials']==1]

author        subreddit  created_utc  score       id  \
1410     gentlyfailing         climateskeptics  1482311670   1.0    dbgineq   
4072     502323                climateskeptics  1416509391   5.0    cm86hqt   
4880     AlyssaMoore           climateskeptics  1297790727   4.0    c1gw4gh   
5066     ActuallyNot           climateskeptics  1399729825   0.0    chef54y   
6818     ultimis               Conservative     1575787989   2.0    fa2xh0f   
...          ...                        ...            ...   ...        ...   
2161715  FormerlyTurnipHugger  climateskeptics  1376678777   4.0    cbp5xxr   
2163604  rojimbo0              climateskeptics  1476655530  -1.0    d8uvl37   
2168758  Maxi720736            climateskeptics  1562409107   3.0    et21vhy   
2169435  Kim147                climateskeptics  1571535204   2.0    f4cxgkt   
2169485  logicalprogressive    climateskeptics  1555550368   3.0    el5ils1   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

In [20]:
nonclimate_data.loc[nonclimate_data['denials']==1].sample(5)

,author,subreddit,created_utc,score,id,body,author_flair_text,type,g_sub,denials
1319206,bingo1952,climateskeptics,1566251240,2.0,exfyea2,"Whoa, Whoa, You said: ""Scenario A is exactly a continuation of the then-current emissions growth trend. It represented a scenario in which nothing ever happened to slow the growth of CO2 emissions from what it was during the 70s and 80s.""\n**THAT HAPPENED in ACTUALITY BUT WORSE, The GHGs were higher than Hansen theorized**\n\nWhy wouldn't we use the projected temperature increase for that scenario to compare to actual temperature increases? As it was the increase in temperature was twice as high as actual measured temperature.\n\nHansen also had a fudge factor built in He had calculated a major volcanic eruption in 2014. That did not happen. However calculating it i reduced Scenario A's temperature projection.\n **TWICE AS MUCH AS REALITY**. You need to quit arguing before I throw you entirely under the bus.\n\nFurthermore the 2017 end date is one that is essentially cherry picked because it is the endpoint where an enso event causes a large excursion upwards to allow alarmists to claim that Scenario B was met. When in fact if the trend rate is measured it can be seen that the temperature per the trend is much lower.\n\n Second, applying a post-hoc bias correction to the forcing ignores the fact that converting GHG increases into forcing is an essential part of the modeling. ESSENTIAL PART OF THE MODELING.\n\n If it was not included to begin with then the modeling was shit. The Scenario that most closely matches reality was Scenario C.\n\nLooked at another way, Scenario A has a slope (post 1988) of about 0.033 while the observed temperatures have a slope of about 0.017. Twice the warming predicted. And that prediction was arbitrarily low due to the missing volcanic eruption and 2. the actual temperatures were measured at a high point in the sinusoidal curve of the normal ups and downs of the temperature measurements. 3. The actual GHGs were higher by 25% than Hansen postulated.\n\nNow do we want to go down the Coulda Woulda Shoulda path in describing the results of the study? I can name 3 different **VALID REASONS** that the projections would be further off in opposition to your assertions. .",NaN,comment,climateskeptic,1.0
1765166,duck_shuck,The_Donald,1544588078,7.0,ebm0dzw,"I mean, how many Americans are clamoring to get into Canada? I’m sure Greenlanders are just fine where they are too. Alaskans only drive through to get to the mainland. Not to mention their immigration policy is (gasp!) merit based and prevents chain migration. They can virtue signal all day long because they don’t have to deal with the problem.",NaN,comment,conspiracysub,1.0
1075614,deck_hand,climateskeptics,1448713362,7.0,cxfnoss,"This is because they know this isn't about CO2. It's about punishing the Western Developed Nations for being economically superior 100 years before the other nations. It's a way of economically reversing history and giving ""developing nations"" the ability to run the world for the next couple of hundred years instead of the ones in charge now.\n\nChina already emits more than 30% of the CO2 emissions on the planet. According to the Climate Alarmists, the US, the EU, Canada, Japan, South Korea, Australia and a few others must reduce our emissions by 80% or the world will die horribly. But, it's okay for China to double their emissions while we cut ours by more than half. It's a given that India and Brazil will double their emissions, too. \n\nLet's say that we cut our emissions back by half, and they double theirs. We will trade places with China, where they are emitting more than 16 Tons per capita while we will emit under 8, but they have 4 times the number of people. They will emit 8 times the amount of GHGs that the US does at that point. But, the alarmists don't want us to drop to half, they want us to drop to 20% of our 2005 levels. That would put us at about half the level, per ca

In [ ]:
def remove_stops(text):
    new_text = " ".join([t for t in text.split(" ") if t not in swords and not t.isdecimal()])
    return new_text


def text_proprocc_stop(text):
    ctext = preprocess_e2e(text)
    stext = remove_stops(ctext)
    if len(stext) > 5:
        return stext
    
    
nonclimate_data['cleaner_text'] = nonclimate_data['body'].parallel_apply(lambda x: text_proprocc_stop(x))

climate_data = climate_data.dropna(subset=['body'])
climate_data['cleaner_text'] = climate_data['body'].parallel_apply(lambda x: text_proprocc_stop(x))

In [ ]:
nonclimate_data.sample(5)

In [ ]:
climate_data = climate_data.dropna(subset=['cleaner_text'])

In [ ]:
nonclimate_data = nonclimate_data.dropna(subset=['cleaner_text'])

In [ ]:
# climcorp = climate_data['cleaner_text'].tolist()
# nonclimcorp = nonclimate_data['cleaner_text'].tolist()

# climcorp_tokens = [c.split() for c in climcorp]
# climcorp_phrases = Phrases(climcorp_tokens, min_count=5, threshold=0.1)
# climcorp_tokes = [climcorp_phrases[t] for t in climcorp_tokens]

# nonclimcorp_tokens = [c.split() for c in nonclimcorp]
# nonclimcorp_phrases = Phrases(nonclimcorp_tokens, min_count=5, threshold=0.1)
# nonlimcorp_tokes = [nonclimcorp_phrases[t] for t in nonclimcorp_tokens]


In [ ]:
# nonlimcorp_tokes[986]

## climate data distributions

In [ ]:
# CC = [" ".join(c) for c in climcorp_tokes]
# CC[0]

In [ ]:
# NC = [" ".join(c) for c in nonlimcorp_tokes]
# NC[0]

## build vocabs for SAGE

In [ ]:
# CC_labs = " .".join(CC)
# NC_labs = " .".join(NC)

# corps = [CC_labs, NC_labs]
# #labs = CC_labs + NC_labs

In [ ]:
precorp = nonclimate_data.groupby('g_sub')['cleaner_text'].apply(list).reset_index()
len(precorp)

In [ ]:
precorp['text'] = precorp['cleaner_text'].apply(lambda x: " .".join(x))
corps = precorp['text'].tolist()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

#vect = CountVectorizer(min_df=2, max_df=0.3)
vect = CountVectorizer()
X = vect.fit_transform(corps)

In [ ]:
vocab = {i:j for j,i in vect.vocabulary_.items()}

In [ ]:
print(len(vocab.keys()))

In [ ]:
precorp['g_sub']

In [ ]:
groups = ['climateskeptic', 'conspiracysub', 'politicssub', 'proclimate']
N = [1 for g in groups]
idxs = np.array([0] + N).cumsum()
x = {sub:np.array(X[start:stop,:].sum(axis=0))[0] for sub,start,stop in zip(groups,idxs[:-1],idxs[1:])}

In [ ]:
x

In [ ]:
value_array = np.array([v for k,v in x.items()])

X_base = value_array.sum(axis=0)

In [ ]:
X_base

In [ ]:
smoothing=1.
mu = np.log(X_base+smoothing) - np.log((X_base+smoothing).sum())

In [ ]:
from generic_func import sage

In [ ]:
import csv
fname = "../lite_data/saged_subvocab.csv"
num_keywords=2000
with open(fname,'w') as fout:
    writer = csv.DictWriter(fout,fieldnames=['source',
                                                   'word',
                                                   'sage',
                                                   'base_count',
                                                   'base_rate',
                                                   'file_count',
                                                   'file_rate'],
                            delimiter='\t'
    )

    etas = defaultdict()

    test_sublist = groups
    #for sub in sublist[:1]:
    for sub in test_sublist:
        try:
            estmt = sage.estimate(x[sub], mu, max_its=1)
            print("worked", sub)
            filename = sub
            eta = estmt

            for word in sage.topK(eta,vocab,num_keywords):
                idx = vect.vocabulary_[word]
                writer.writerow({'source':filename,
                                 'word':word,
                                 'sage':eta[idx],
                                 'file_count':x[filename][idx],
                                 'file_rate':x[filename][idx]/float(x[filename].sum()),
                                 'base_count':X_base[idx],
                                 'base_rate':X_base[idx]/float(X_base.sum())
                })
                
        except:
            print("doesnt work", sub)


In [ ]:
sgwords = pd.read_csv("../lite_data/saged_subvocab.csv", sep='\t', header=None)
sgwords.head()

In [ ]:
sgwords.loc[sgwords[0]=='climateskeptic'].head(50)

In [ ]:
sgwords.loc[sgwords[0]=='non_climate'].head(50)